# Lesson 17. Practice with some advanced SimPy constructs

### SA421 Fall 2015

**Problem.**
Chef Fulkerson sells her famous dark chocolate peanut butter salted
caramel truffles from her food truck. 
Customers arrive at the truck according to an exponential interarrival
time distribution with a mean of 5 minutes. 
Due to the popularity of these truffles, there is a limit of 5 per
customer. 
Each customer orders 1, 2, 3, 4, or 5 according to the following
probability distribution:

| Number of truffles ordered | 1   | 2   | 3   | 4   | 5   |
|----------------------------------------------------------|
| Probability                | 0.1 | 0.1 | 0.2 | 0.2 | 0.4 |

If a customer wants more truffles than what is available, he or she
simply buys the remaining truffles.

Since the truffles are made to order, serving a customer takes
between 3 and 9 minutes, uniformly distributed.
Customers are somewhat impatient, and will abandon the line without getting served if they've waited too long.
The time that a customer will spend in line before abandoning it is exponentially distributed with mean 10 minutes.


Chef Fulkerson starts operating her truck at 10:00, and stays open until she sells out.


* Model the operation of this system in SimPy.


* Using 100 replications:
    - Estimate the **stockout time**, i.e. when Chef Fulkerson runs out of truffles.
    - Estimate the fraction of customers who abandon the line without getting served.

*Hints.* You can run a simulation without a predefined end time using `simulate(until = inf)`. You can stop a simulation anytime using `stopSimulation()`. [Here](http://simpy.sourceforge.net/old/SManual/SManual.html#simulation-with-simpy) is the relevant part of the documentation.

### Setup

In [1]:
##### Setup #####
# Import various functions from NumPy
from numpy import inf, mean

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, rand, exponential, uniform

# Import bisect_left from bisect
from bisect import bisect_left

# Import everything from SimPy
from SimPy.Simulation import *

### Model

In [2]:
##### Parameters #####
class P:
    # Customers arrive according to an exponential interarrival
    # time distribution with mean 5 minutes
    interarrivalTimeMean = 5
    
    # Service time is uniformly distributed between 3 and 9 minutes
    serviceTimeMin = 3
    serviceTimeMax = 9
    
    # Reneging times are exponentially distributed with mean 10 minutes
    renegeTimeMean = 10
    
    # Number of truffles available at the start
    nTruffles = 200

    
##### Distributions #####
class D:
    # Number of truffles per customer
    def trufflesPerCustomer():
        # Sorted list of possible values and -inf
        a = [-inf, 1, 2, 3, 4, 5]

        # cdf at each possible value
        cdf = [0, 0.1, 0.2, 0.4, 0.6, 1.0]

        # Generate variate
        variate = a[bisect_left(cdf, rand())]

        # Return generated variate
        return variate

    
##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Determine how many truffles the customer wants
        trufflesWanted = D.trufflesPerCustomer()
        
        # Customer arrives, joins queue, waits until service 
        # or renege time limit, whichever comes first        
        renegeTime = exponential(scale = P.renegeTimeMean)
        yield (request, self, R.server), (hold, self, renegeTime)
            
        # Customer doesn't renege...
        if self.acquired(R.server):
            M.renege.observe(0)
                
            # Customer is released from queue, starts service
            serviceTime = uniform(low = P.serviceTimeMin, high = P.serviceTimeMax)
            yield hold, self, serviceTime
        
            # Customer gets truffles
            if trufflesWanted > L.truffles.amount:
                yield get, self, L.truffles, L.truffles.amount
            else:
                yield get, self, L.truffles, trufflesWanted

            # Customer finishes service, leaves
            yield release, self, R.server

            # Check for stockout
            if L.truffles.amount == 0:
                M.stockoutTime.observe(now())
                stopSimulation()
                      
        # Customer reneges...
        else:
            M.renege.observe(1)
            
# Entrance
class Entrance(Process):
    def behavior(self):
        # Start customer for number of customers
        nCustomers = 0
        
        while True:
            # Interarrival time
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
 
            # Create customer using the template defined in the Customer class
            c = Customer(name = "Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())
            
            # Count this new customer
            nCustomers += 1

            
##### Resources #####
class R:
    server = None   

    
##### Levels #####
class L:
    truffles = None
    
    
##### Monitors #####
class M:
    renege = None
    stockoutTime = None
    
            
##### Simulation #####
def model(inputSeed):
    # Initialize SimPy
    initialize()
    
    # Initialize a seed for the random number generator
    seed(inputSeed)

    # Create server resource
    R.server = Resource(capacity = 1, monitored = True)

    # Create level for truffles
    L.truffles = Level(initialBuffered = P.nTruffles)
    
    # Create monitor for reneging customers
    M.renege = Monitor()
    
    # Create monitor for stock out time
    M.stockoutTime = Monitor()

    # Activate the entrance
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = inf)

    # Compute stockout time - this monitor should only have one observation
    stockoutTime = M.stockoutTime.mean()
    
    # Compute fraction of customers who reneged
    fracRenege = M.renege.mean()
    
    # Return stockout time and fraction of customers who reneged
    return [stockoutTime, fracRenege]

### Experiment

In [3]:
# Number of replications 
n = 100

# Collect observations
stockoutTimeObs = []
fracRenegeObs = []
for i in range(n):
    stockoutTime, fracRenege = model(123*i)
    stockoutTimeObs.append(stockoutTime)
    fracRenegeObs.append(fracRenege)
    
# Observed sample mean
stockoutTimeSM = mean(stockoutTimeObs)
fracRenegeSM = mean(fracRenegeObs)

# Print estimated stockout time
print("Estimated stockout time: {0} hours after 10:00".format(stockoutTimeSM / 60))

# Print estimated fraction of customers who reneged
print("Estimated fraction of customers who reneged: {0}".format(fracRenegeSM))

Estimated stockout time: 6.992636857316447 hours after 10:00
Estimated fraction of customers who reneged: 0.3427228052888532
